In [1]:
import numpy as np
import cv2
import os, sys
import matplotlib.pyplot as plt

sys.path.append(os.path.join("/data0/wxy/3d_pose/stereo-estimation/"))
from lib.dataset.human36m import Human36MMultiViewDataset


h36m_root = os.path.join("/data1/share/dataset/human36m_multi-view/", "processed")
labels_stereo_npy_path = "/data1/share/dataset/human36m_multi-view/extra/human36m-stereo-labels-GTbboxes.npy"

dataset = Human36MMultiViewDataset(
    h36m_root,
    labels_stereo_npy_path,
    train=True,                       # include all possible data
    test=True,
    image_shape=None,                 # don't resize
    retain_every_n_frames_in_test=1,  # yes actually ALL possible data
    with_damaged_actions=True,        # I said ALL DATA
    kind="mpii",
    norm_image=False,                 # don't do unnecessary image processing
    undistort_images=True,                 
    crop=True)                       # don't crop
print("Dataset length:", len(dataset))

def test_rectificate(idx):
    sample = dataset[idx]
    
    shot = dataset.labels['table'][idx]
    subject_idx = shot['subject_idx']
    action_idx  = shot['action_idx']
    frame_idx   = shot['frame_idx']

    subject = dataset.labels['subject_names'][subject_idx]
    action = dataset.labels['action_names'][action_idx]

    available_cameras = list(range(len(dataset.labels['action_names'])))
    
    for camera_idx, bbox in enumerate(shot['bbox_by_camera_tlbr']):
        if bbox[2] == bbox[0]: # bbox is empty, which means that this camera is missing
            available_cameras.remove(camera_idx)
    
    for camera_idx, image, keypoit_2d in zip(available_cameras, sample['images'], sample['keypoints_2d']):
        camera_name = dataset.labels['camera_names'][camera_idx]

        output_image_folder = os.path.join(camera_name)
        output_image_path = os.path.join(output_image_folder, 'img_%06d.png' % (idx+1))
        os.makedirs(output_image_folder, exist_ok=True)

        fig = plt.figure()
        plt.imshow(image)
        plt.scatter(keypoit_2d[:, 0], keypoit_2d[:, 1], s=1, c='red')
        plt.savefig(output_image_path)
        plt.close()

for idx in range(0, 160000, 5000):
    test_rectificate(idx)

/data0/wxy/anaconda3/envs/py37/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset length: 161362
